In [18]:
import pandas as pd
from datasets import Dataset
import json

# Load data from JSON file
with open('data/new.json', 'r') as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)

# Ensure correct columns
assert 'text' in df.columns and 'label' in df.columns, "Data must contain 'text' and 'label' columns"

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)


In [19]:
import os
print(os.getcwd())


/home/shivv/Desktop/sih/SIH_2024/ipc_nlp


In [20]:
from transformers import RobertaTokenizer

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Preprocess function
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

# Tokenize dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)


/home/shivv/Desktop/sih/SIH_2024/ipc_nlp/env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/127 [00:00<?, ? examples/s]

In [21]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128,
        clean_up_tokenization_spaces=False  # Explicitly set this to control behavior
    )


In [28]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import json

# Load model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(df['label'].unique()))

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # or use `eval_strategy` in newer versions
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',  # Specify a logging directory if desired
)

# Create Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset  # Ideally, use a separate validation set
)

# Train the model
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')


In [34]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import json

# Load data from JSON file
with open('data/new.json', 'r') as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)

# Ensure 'text' and 'label' columns are present
assert 'text' in df.columns and 'label' in df.columns, "Data must contain 'text' and 'label' columns"

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(df['label'].unique()))

# Preprocess the data
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Create Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset  # Ideally, use a separate validation set
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

# Make predictions
from transformers import pipeline

nlp = pipeline('text-classification', model='./model', tokenizer='./model')
text = "Your input text here"
print(nlp(text))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/127 [00:00<?, ? examples/s]

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [36]:
!pip cache purge


Files removed: 20


In [42]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from datasets import Dataset as HF_Dataset
import json

# Load model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(df['label'].unique()))

# Load and preprocess data
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

# Load JSON data
with open('data/new.json', 'r') as file:
    data = json.load(file)

# Convert to Hugging Face Dataset
dataset = HF_Dataset.from_dict(data)
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Custom Dataset class for PyTorch
class CustomDataset(Dataset):
    def __init__(self, encoded_dataset):
        self.dataset = encoded_dataset
        self.features = {k: torch.tensor(v) for k, v in zip(self.dataset.column_names, zip(*self.dataset.values()))}

    def __len__(self):
        return len(self.features['input_ids'])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.features.items()}

# Create DataLoader
train_dataset = CustomDataset(encoded_dataset)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Setup optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 3  # Number of training steps
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
model.train()
for epoch in range(3):
    for batch in train_loader:
        inputs = {key: val.to(device) for key, val in batch.items() if key in tokenizer.model_input_names}
        labels = batch.get('labels', None)  # Ensure 'labels' are correctly retrieved

        if labels is not None:
            labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'list' object has no attribute 'items'

In [51]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset as HF_Dataset
import json

# Load the data
with open('data/new.json', 'r') as file:
    data = json.load(file)

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Ensure 'text' and 'label' columns are present
assert 'text' in df.columns and 'label' in df.columns, "Data must contain 'text' and 'label' columns"

# Convert the DataFrame to a Hugging Face Dataset
dataset = HF_Dataset.from_pandas(df)

# Load Tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Define the preprocessing function
def preprocess_function(examples):
    # Print the structure of `examples` to debug
    print("Type of examples:", type(examples))
    print("Examples received by preprocess_function:", examples)
    
    if isinstance(examples, dict):
        # Single example
        if 'text' in examples:
            return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
        else:
            raise ValueError("Expected 'text' key in the input dictionary.")
    
    elif isinstance(examples, list):
        # List of examples
        if len(examples) > 0 and isinstance(examples[0], dict):
            # List of dictionaries
            texts = [ex['text'] for ex in examples if 'text' in ex]
            return tokenizer(texts, truncation=True, padding='max_length', max_length=128)
        else:
            raise ValueError("The list should contain dictionaries with 'text' key.")
    
    else:
        raise ValueError("The input is neither a dictionary nor a list of dictionaries.")

# Apply the preprocessing function to the dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Check the output to ensure it's correctly formatted
print("Example data after tokenization:", encoded_dataset[0])

# Define a custom Dataset class for PyTorch
class CustomDataset(Dataset):
    def __init__(self, encoded_dataset):
        self.input_ids = torch.tensor(encoded_dataset['input_ids'], dtype=torch.long)
        self.attention_mask = torch.tensor(encoded_dataset['attention_mask'], dtype=torch.long)
        self.labels = torch.tensor(encoded_dataset['label'], dtype=torch.long)  # Ensure labels are in tensor format

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Create DataLoader
train_dataset = CustomDataset(encoded_dataset)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Load the model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(df['label'].unique()))

# Setup optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 3  # Number of training steps
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
model.train()
for epoch in range(3):
    for batch in train_loader:
        # Move tensors to the device
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")


Map:   0%|          | 0/127 [00:00<?, ? examples/s]

Type of examples: <class 'datasets.formatting.formatting.LazyBatch'>
Examples received by preprocess_function: {'text': ['This Act shall be called the Indian Penal Code, and shall extend to the whole of India except the State of Jammu and Kashmir.', 'Every person shall be liable to punishment under this Code and not otherwise for every act or omission contrary to the provisions thereof, of which he shall be guilty within India.', 'Any person liable, by any Indian law, to be tried for an offence committed beyond India shall be dealt with according to the provisions of this Code for any act committed beyond India in the same manner as if such act had been committed within India.', 'The provisions of this Code apply also to any offence committed by:', 'Nothing in this Act shall affect the provisions of any Act for punishing mutiny and desertion of officers, soldiers, sailors or airmen in the service of the Government of India or the provision of any special or local law.', 'Throughout thi

ValueError: The input is neither a dictionary nor a list of dictionaries.

In [52]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset as HFDataset

# Load and prepare data
data = {
    'text': [
        "Sample text for classification.",
        "Another sample text."
    ],
    'label': [1, 0]
}

df = pd.DataFrame(data)

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def preprocess_function(examples):
    # Ensure that `examples` is either a list of dicts or a dict
    if isinstance(examples, dict):
        if 'text' in examples:
            return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
        else:
            raise ValueError("Expected 'text' key in the input dictionary.")
    
    elif isinstance(examples, list):
        if len(examples) > 0 and isinstance(examples[0], dict):
            texts = [ex['text'] for ex in examples if 'text' in ex]
            return tokenizer(texts, truncation=True, padding='max_length', max_length=128)
        else:
            raise ValueError("The list should contain dictionaries with 'text' key.")
    
    else:
        raise ValueError("The input is neither a dictionary nor a list of dictionaries.")

# Convert DataFrame to Hugging Face Dataset
dataset = HFDataset.from_pandas(df)

# Apply the preprocessing function
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, encoded_dataset):
        self.input_ids = torch.tensor(encoded_dataset['input_ids'], dtype=torch.long)
        self.attention_mask = torch.tensor(encoded_dataset['attention_mask'], dtype=torch.long)
        self.labels = torch.tensor(encoded_dataset['label'], dtype=torch.long)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Create DataLoader
train_dataset = CustomDataset(encoded_dataset)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Load the model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(df['label'].unique()))
model.to(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

# Define optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

# Training loop
for epoch in range(3):
    model.train()
    for batch in train_loader:
        inputs = {
            'input_ids': batch['input_ids'].to(model.device),
            'attention_mask': batch['attention_mask'].to(model.device)
        }
        labels = batch['labels'].to(model.device)
        
        optimizer.zero_grad()
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Optional: Using Hugging Face Trainer API for convenience
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset
)

trainer.train()


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

ValueError: The input is neither a dictionary nor a list of dictionaries.